In [1]:
import torch as t
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import json
import random

In [2]:
with open('common_words_no_names.txt') as f:
    common_words = f.read().splitlines()
g = t.Generator().manual_seed(214743647)
numerated = {}
for i in range(len(common_words)):
    numerated[common_words[i]] = i

lookup = t.randn((len(common_words), 10), generator = g)
lookup1 = lookup.clone().detach()
common_words = set(common_words)

In [3]:
model_name = 'gpt2'  
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name, clean_up_tokenization_spaces = False)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [4]:
def get_prob(text):
    if ' ' not in text:
        return t.tensor(0)
    input_ids = tokenizer.encode(text, return_tensors='pt')
    prob = 1
    with t.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits
    for i in range(list(t.Tensor.size(input_ids))[1] - 1):
        next_token_logits = logits[:, i, :]
        probabilities = F.softmax(next_token_logits, dim=-1)
        next_token_id = input_ids[0][i + 1].item()
        prob *= probabilities[0][next_token_id]
    if prob != 1:
        return prob
    else:
        return t.tensor(0)

In [5]:
class A:
    def __init__(self):
        self.common_words = open('common_words_no_names.txt').read().split()
        with open('memo_dict.json', 'r') as f:
            self.memoDict = json.load(f)

    def save_anagrams(self):
        open('memo_dict.json', 'r').close()
        with open('memo_dict.json', 'w') as f:
            json.dump(self.memoDict, f)

    def anagwams(self, word):
        if word == '':
            return ['']
        if word in self.memoDict:
            return self.memoDict[word]
        dict = {}
        for w in common_words:
            wordHusk = word
            included = True
            for c in w:
                if c not in wordHusk:
                    included = False
                    break
                else:
                    i = wordHusk.index(c)
                    wordHusk = wordHusk[:i] + wordHusk[i+1:]
            if included:
                dict[w] = wordHusk
        if len(word) < self.length - 1:
                self.memoDict[word] = [w + ' ' + g for w in list(dict.keys()) for g in self.anagwams(dict[w])]
        return [w + ' ' + g for w in list(dict.keys()) for g in self.anagwams(dict[w])]

    def anagrams(self, input_word):
        self.length = len(input_word)
        input_word = input_word.lower()
        word = ''
        # for c in input_word:
        #     if c in 'abcdefghijklmnopqrstuvwxyz':
        #         word += c
        words = self.anagwams(input_word.lower())
        for w in words:
            w = w.split(' ')
        return "\n".join(sorted(words, reverse = True, key = get_prob))

In [6]:
g = A()
print(g.anagrams('testingit')) # initial: 13.1; after: 12.6, 11.9

testing it 
setting it 
it testing 
it setting 
tin gets it 
tin get its 
tin it gets 
get tits in 
tin its get 
gets tit in 
tin get sit 
get in tits 
tn it is get 
tn is get it 
tin get ist 
in get tits 
gets in tit 
tin ist get 
tin sit get 
tn i get its 
gets tin it 
tn i gets it 
sit tin get 
sin get tit 
tn get is it 
tn get it is 
in tits get 
its tin get 
ins get tit 
tn is it get 
tn its i get 
gets it tin 
get tit ins 
sin tit get 
sit get tin 
ins tit get 
ist tin get 
tn sit i get 
titten sig 
its get tin 
tn i it gets 
get tin its 
it sig tent 
it tent sig 
ist get tin 
it gets tin 
get tit sin 
tits get in 
it tin gets 
get sin tit 
in tit gets 
it gis tent 
get tin sit 
get its tin 
get sit tin 
tn it gets i 
in gets tit 
tn it si get 
ten sig tit 
get tin ist 
tn it get is 
tn it i gets 
tn get i its 
get ins tit 
ten tit sig 
tn i get sit 
tn i get ist 
tn get its i 
tn get i sit 
tit gets in 
tn si get it 
get ist tin 
get tn it is 
tn ist i get 
net tit sig 
tn i its

In [7]:
text = "Before the towers"
input_ids = tokenizer.encode(text, return_tensors='pt')
with t.no_grad():
    outputs = model(input_ids)
    logits = outputs.logits

# Get logits for the last token in the input sequence
next_token_logits = logits[:, -1, :]

# Convert logits to probabilities
probabilities = F.softmax(next_token_logits, dim=-1)

# Get the top 10 most probable tokens
top_k = 10
top_k_probs, top_k_indices = t.topk(probabilities, top_k)

top_k_tokens = tokenizer.decode(top_k_indices[0]).split()
top_k_probabilities = top_k_probs[0].tolist()

for token, prob in zip(top_k_tokens, top_k_probabilities):
    i = 0
    #print(f"Token: {token}, Probability: {prob:.4f}")
    